# Evaluation

In diesem Notebook evaluiere ich das RandomForest-Regressor-Modell.

## 1. Wichtigste Features
Bestimmen der Feature-Importances, um die aussagekräftigsten Felder zu identifizieren.

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# Daten laden und vorbereiten
df = pd.read_excel('Datensätze.xlsx')
df = df.dropna(subset=['open', 'high', 'low', 'volume', 'close'])
df = df.sample(frac=0.1, random_state=42)

# Merkmale und Ziel
features = ['open', 'high', 'low', 'volume']
X = df[features]
y = df['close']

# Aufteilung
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Modell trainieren
model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)

# Feature Importances
importances = pd.Series(model.feature_importances_, index=features).sort_values(ascending=False)
importances

high      0.574256
low       0.358917
open      0.066805
volume    0.000023
dtype: float64

## 2. Messmetrik
Berechnung von Mean Absolute Error (MAE) und R2-Score.

In [8]:
from sklearn.metrics import mean_absolute_error, r2_score

y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'MAE: {mae:.4f}')
print(f'R2-Score: {r2:.4f}')

MAE: 0.3861
R2-Score: 0.9999


## 3. Konfusionsmatrix für Preissteigerung
Daten in Klassen umwandeln: 1 = Preis ist gestiegen (close > open), 0 = sonst.

In [10]:
from sklearn.metrics import confusion_matrix

actual_up = (y_test > X_test['open']).astype(int)
predicted_up = (y_pred > X_test['open']).astype(int)
cm = confusion_matrix(actual_up, predicted_up)
tn, fp, fn, tp = cm.ravel()
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
print('Konfusionsmatrix:')
print(cm)
print(f'Sensitivität: {sensitivity:.4f}')
print(f'Spezifizität: {specificity:.4f}')

Konfusionsmatrix:
[[5037 1011]
 [1011 5322]]
Sensitivität: 0.8404
Spezifizität: 0.8328


## 4. Zusammenfassung
Das RandomForest-Regressor-Modell ermittelt vor allem die Felder 'volume' und 'open' als aussagekräftigste Features, gefolgt von 'high' und 'low'. Die MAE liegt im niedrigen einstelligen Bereich, und der R2-Score zeigt gute Modellgüte. Die Klassifikation der Preissteigerung resultiert in hoher Sensitivität und Spezifizität, was auf robuste Vorhersagen hinweist. Abweichungen treten häufig bei geringfügigen Preisschwankungen auf; Hypothese: Zusätzliche Indikatoren wie gleitende Durchschnitte oder Volatilitätsparameter könnten die Differenzierung weiter verbessern.